In [53]:
import pandas as pd
import numpy as np

from dataprep.eda import plot

# Required for 3A (Distance Travelled)
import geopy.distance
import geocoder

In [9]:
df19 = pd.read_csv('../data/Jan_2019_ontime.csv')
df20 = pd.read_csv('../data/Jan_2020_ontime.csv')

df19['Year'] = 2019
df20['Year'] = 2020

print(df19.shape)
print(df20.shape)

(583985, 23)
(607346, 23)


In [14]:
# find a better way to compare the 2x DFs . 
for i in range(len(df19.columns)):
    if df19.columns[i] == df20.columns[i]:
        print("Match")
    else:
        print("NO MATCH")

Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match


In [44]:
# reset here
df = pd.concat([df19, df20], ignore_index=True, sort=False)

In [45]:
df.shape

(1191331, 23)

In [46]:
df_airport = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', \
                     header=None)
df_airport.columns=['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz database time zone','Type','Source']


In [47]:
df_airport.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [48]:
df_ap = df_airport[['Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone']]

In [49]:
df.columns

Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'CANCELLED',
       'DIVERTED', 'DISTANCE', 'Unnamed: 21', 'Year'],
      dtype='object')

In [50]:
df_ap.head(1)

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone
0,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10


In [51]:
df = df.merge(df_ap, how='left', left_on='ORIGIN', right_on='IATA')
df = df.merge(df_ap, how='left', left_on='DEST', right_on='IATA')

In [52]:
df.columns


Index(['DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN',
       'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST', 'DEP_TIME',
       'DEP_DEL15', 'DEP_TIME_BLK', 'ARR_TIME', 'ARR_DEL15', 'CANCELLED',
       'DIVERTED', 'DISTANCE', 'Unnamed: 21', 'Year', 'Name_x', 'City_x',
       'Country_x', 'IATA_x', 'ICAO_x', 'Latitude_x', 'Longitude_x',
       'Altitude_x', 'Timezone_x', 'Name_y', 'City_y', 'Country_y', 'IATA_y',
       'ICAO_y', 'Latitude_y', 'Longitude_y', 'Altitude_y', 'Timezone_y'],
      dtype='object')

In [94]:
#Iterating through all deliveries to find the total distance travelled (as the crow flies)
distances = range(len(df))
dist_lst = []
cnt = 0
for i in distances:
    #if ((df['ORIGIN'].iloc[i].isna() ==True) | (df['DEST'].iloc[i].isna() ==True)):
    if (pd.isnull(df['Latitude_x']) != True):
        if pd.isnull(df['Latitude_x']) != True :
            origin = (df['Latitude_x'].iloc[i] , df['Longitude_x'].iloc[i])
            dest = (df['Latitude_y'].iloc[i] , df['Longitude_y'].iloc[i])
            dist_lst.append(round(geopy.distance.distance(origin, dest).km,2))
    else:
        cnt += 1
        pass

print(cnt)
df['distance'] = dist_lst  

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [87]:
df['Latitude_x'].iloc[595921] == Nan


NameError: name 'Nan' is not defined

In [61]:
df.ORIGIN[df.Latitude_x.isna() == True].unique()

array(['EAR', 'XWA'], dtype=object)

In [77]:
df[df.ORIGIN == 'XWA']

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,Timezone_x,Name_y,City_y,Country_y,IATA_y,ICAO_y,Latitude_y,Longitude_y,Altitude_y,Timezone_y
595921,1,3,OO,20304,OO,N477CA,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
596066,1,3,OO,20304,OO,N8942A,4007,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
597139,1,3,OO,20304,OO,N157SY,5981,16869,1686901,XWA,...,NaN,Denver International Airport,Denver,United States,DEN,KDEN,39.861698,-104.672997,5431.0,-7
616207,2,4,OO,20304,OO,N809SK,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
616414,2,4,OO,20304,OO,N685BR,4007,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143234,29,3,OO,20304,OO,N145SY,5817,16869,1686901,XWA,...,NaN,Denver International Airport,Denver,United States,DEN,KDEN,39.861698,-104.672997,5431.0,-7
1163677,30,4,OO,20304,OO,N549CA,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
1163933,30,4,OO,20304,OO,N455SW,4007,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6
1178249,31,5,OO,20304,OO,N820SK,3798,16869,1686901,XWA,...,NaN,Minneapolis-St Paul International/Wold-Chamber...,Minneapolis,United States,MSP,KMSP,44.882000,-93.221802,841.0,-6


In [76]:
df_ap[df_ap.IATA == 'JFK']

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone


In [68]:
df_ap[df_ap.IATA == 'EAR']

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone


In [69]:
df_ap[df_ap.IATA == 'XWA']

,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone


In [13]:
df19.columns

'DAY_OF_MONTH'

In [12]:
if df19.columns == df20.columns:
    print("Cols the same")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()